In [17]:
from dotenv import load_dotenv
from langchain_core.messages import SystemMessage
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_openai import ChatOpenAI

load_dotenv()

True

## cell to check db connection with llm

In [18]:

db = SQLDatabase.from_uri("sqlite:///advisors.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Advisors LIMIT 10;")

sqlite
['advisors', 'brokercheck_data', 'linkedin_profiles']


"[(1, 1000034, None, 'Stephen', 'Wendell', 'Bryars', None, None, 'Mml Investors Services, Llc', '10409', '7.077', '2017-03-25', 'Mml Investors Services, Llc', '10409', '7.077', '2017-03-25', '3 West Garden Street Suite 510', 'Pensacola', 'FL', '32502', 'Pensacola-Ferry Pass-Brent, FL', 'SIE,Series 22,Series 6,Series 63,Series 65', 'Investment Advisor Representative', None, None, None, None, 'sbryars@bryarsfinancialgroup.com', None, None, None, 'https://strategicfinancialpartners.com/our-team/stephenbryars', 43, None, 'Msi Financial Services, Inc.: 01/02/2015 - 03/25/2017, New England Securities: 08/10/1981 - 01/02/2015', 'Msi Financial Services, Inc.: 01/02/2015 - 03/25/2017, New England Securities: 08/10/1981 - 01/02/2015', 'male', 1092980425364, None, None, None, None, None, None, None, None, None, None, None, 'eMoney,SEI Investments,Black Diamond,Assetmark,Brinker Capital,Envestnet', 'Nitrogen (Riskalyze),Smarsh,Carta,Broadridge,Factset,DocuSign,SmartAsset,FMG Suite,Calendly,RightCa

In [22]:
db.get_context()

{'table_info': '\nCREATE TABLE advisors (\n\tid INTEGER NOT NULL, \n\tcrd INTEGER, \n\tnpn FLOAT, \n\tfirst_name VARCHAR, \n\tmiddle_name VARCHAR, \n\tlast_name VARCHAR, \n\tother_names TEXT, \n\tteam FLOAT, \n\tbroker_dealer VARCHAR, \n\tbroker_dealer_crd VARCHAR, \n\tyears_with_current_bd VARCHAR, \n\tcurrent_bd_start_date VARCHAR, \n\tria VARCHAR, \n\tria_crd VARCHAR, \n\tyears_with_current_ria VARCHAR, \n\tcurrent_ria_state_date VARCHAR, \n\taddress VARCHAR, \n\tcity VARCHAR, \n\tstate VARCHAR, \n\tzip VARCHAR, \n\tmetro_area VARCHAR, \n\tlicenses_exams VARCHAR, \n\ttitle VARCHAR, \n\tdesignations VARCHAR, \n\tphone VARCHAR, \n\tphone_type VARCHAR, \n\tlinkedin VARCHAR, \n\temail_1 VARCHAR, \n\temail_2 VARCHAR, \n\temail_3 VARCHAR, \n\tpersonal_email VARCHAR, \n\tbio VARCHAR, \n\tyears_of_experience INTEGER, \n\test_age FLOAT, \n\tprevious_broker_dealer VARCHAR, \n\tprevious_ria VARCHAR, \n\tgender VARCHAR, \n\tteam_id INTEGER, \n\tperson_tag_role FLOAT, \n\tperson_tag_family VARCH

In [ ]:
print(db.get_usable_table_names())
print(db.table_info)


In [19]:
llm = ChatOpenAI(model="gpt-4o")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

### Query Validation

In [23]:
from langchain.chains import create_sql_query_chain


chain = create_sql_query_chain(llm, db)
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

In [25]:
context = db.get_context()

prompt_with_context = chain.get_prompts()[0].partial(table_info=context["table_info"])

In [60]:
context["table_info"]

'\nCREATE TABLE advisors (\n\tid INTEGER NOT NULL, \n\tcrd INTEGER, \n\tnpn FLOAT, \n\tfirst_name VARCHAR, \n\tmiddle_name VARCHAR, \n\tlast_name VARCHAR, \n\tother_names TEXT, \n\tteam FLOAT, \n\tbroker_dealer VARCHAR, \n\tbroker_dealer_crd VARCHAR, \n\tyears_with_current_bd VARCHAR, \n\tcurrent_bd_start_date VARCHAR, \n\tria VARCHAR, \n\tria_crd VARCHAR, \n\tyears_with_current_ria VARCHAR, \n\tcurrent_ria_state_date VARCHAR, \n\taddress VARCHAR, \n\tcity VARCHAR, \n\tstate VARCHAR, \n\tzip VARCHAR, \n\tmetro_area VARCHAR, \n\tlicenses_exams VARCHAR, \n\ttitle VARCHAR, \n\tdesignations VARCHAR, \n\tphone VARCHAR, \n\tphone_type VARCHAR, \n\tlinkedin VARCHAR, \n\temail_1 VARCHAR, \n\temail_2 VARCHAR, \n\temail_3 VARCHAR, \n\tpersonal_email VARCHAR, \n\tbio VARCHAR, \n\tyears_of_experience INTEGER, \n\test_age FLOAT, \n\tprevious_broker_dealer VARCHAR, \n\tprevious_ria VARCHAR, \n\tgender VARCHAR, \n\tteam_id INTEGER, \n\tperson_tag_role FLOAT, \n\tperson_tag_family VARCHAR, \n\tperson_

In [26]:

examples = [
    {
        "input": "Tell me more about Stephen Bryars?",
        "query": "SELECT * FROM advisors WHERE first_name = 'Stephen' AND last_name = 'Bryars';"
    },
    {
        "input": "What schools did Douglas Gaines go to?",
        "query": "SELECT linkedin_profiles.educations FROM linkedin_profiles JOIN advisors ON linkedin_profiles.advisor_id = advisors.id WHERE advisors.first_name = 'Douglas' AND advisors.last_name = 'Gaines';"
    },
    {
        "input": "Do they have any disclosure? If so, how recent and what was the dollar amount?",
        "query": "SELECT disclosures FROM brokercheck_data JOIN advisors ON brokercheck_data.advisor_id = advisors.id WHERE advisors.first_name = 'James' AND advisors.last_name = 'Myatt';"
    },
    {
        "input": "Who does James Myatt work for?",
        "query": "SELECT broker_dealer, ria FROM advisors WHERE first_name = 'James' AND last_name = 'Myatt';"
    },
    {
        "input": "How many people work at Stifel, Nicolaus & Company, Incorporated?",
        "query": "SELECT COUNT(*) FROM advisors WHERE firm_company_name = 'Stifel, Nicolaus & Company, Incorporated';"
    },
    {
        "input": "Find the LinkedIn profile of George Damasco.",
        "query": "SELECT * FROM linkedin_profiles JOIN advisors ON linkedin_profiles.advisor_id = advisors.id WHERE advisors.first_name = 'George' AND advisors.last_name = 'Damasco';"
    },
    {
        "input": "List all advisors in Bellevue, WA.",
        "query": "SELECT * FROM advisors WHERE city = 'Bellevue' AND state = 'WA';"
    },
    {
        "input": "What are the previous broker-dealer and ria for Thomas Hoffman?",
        "query": "SELECT previous_broker_dealer, previous_ria FROM advisors WHERE first_name = 'Thomas' AND last_name = 'Hoffman';"
    },
    {
        "input": "How many years of experience does Stephen Bryars have?",
        "query": "SELECT years_of_experience FROM advisors WHERE first_name = 'Stephen' AND last_name = 'Bryars';"
    },
    {
        "input": "What licenses and exams does Douglas Gaines have?",
        "query": "SELECT licenses_exams FROM advisors WHERE first_name = 'Douglas' AND last_name = 'Gaines';"
    },
    {
        "input": "Find all LinkedIn profiles of advisors who are not open to work.",
        "query": "SELECT * FROM linkedin_profiles WHERE isOpenToWork = FALSE;"
    },
    {
        "input": "List all advisors who have worked for more than 10 years with their current broker-dealer.",
        "query": "SELECT * FROM advisors WHERE years_with_current_bd > 10;"
    },
    {
        "input": "What is the estimated age of James Myatt?",
        "query": "SELECT est_age FROM advisors WHERE first_name = 'James' AND last_name = 'Myatt';"
    },
    {
        "input": "How many advisors work in Florida?",
        "query": "SELECT COUNT(*) FROM advisors WHERE state = 'FL';"
    },
    {
        "input": "List all advisors who have a LinkedIn profile picture.",
        "query": "SELECT * FROM linkedin_profiles WHERE profilePicture IS NOT NULL;"
    }
]

In [27]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

example_prompt = PromptTemplate.from_template("User input: {input}\nSQL query: {query}")
prompt = FewShotPromptTemplate(
    examples=examples[:5],
    example_prompt=example_prompt,
    prefix="You are a SQLite expert. Given an input question, create a syntactically correct SQLite query to run. Unless otherwise specificed, do not return more than {top_k} rows.\n\nHere is the relevant table info: {table_info}\n\nBelow are a number of examples of questions and their corresponding SQL queries.",
    suffix="User input: {input}\nSQL query: ",
    input_variables=["input", "top_k", "table_info"],
)

In [29]:
print(prompt.format(input="How many advisors are there?", top_k=1, table_info="advisor"))

You are a SQLite expert. Given an input question, create a syntactically correct SQLite query to run. Unless otherwise specificed, do not return more than 1 rows.

Here is the relevant table info: advisor

Below are a number of examples of questions and their corresponding SQL queries.

User input: Tell me more about Stephen Bryars?
SQL query: SELECT * FROM advisors WHERE first_name = 'Stephen' AND last_name = 'Bryars';

User input: What schools did Douglas Gaines go to?
SQL query: SELECT linkedin_profiles.educations FROM linkedin_profiles JOIN advisors ON linkedin_profiles.advisor_id = advisors.id WHERE advisors.first_name = 'Douglas' AND advisors.last_name = 'Gaines';

User input: Do they have any disclosure? If so, how recent and what was the dollar amount?
SQL query: SELECT disclosures FROM brokercheck_data JOIN advisors ON brokercheck_data.advisor_id = advisors.id WHERE advisors.first_name = 'James' AND advisors.last_name = 'Myatt';

User input: Who does James Myatt work for?
SQL 

In [33]:
from langchain_community.vectorstores import FAISS
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    FAISS,
    k=5,
    input_keys=["query"],
)

In [36]:
chain = create_sql_query_chain(llm, db, prompt)
chain.invoke({"question": "what school did Bill Stropp go to?"})

"```sql\nSELECT linkedin_profiles.educations \nFROM linkedin_profiles \nJOIN advisors ON linkedin_profiles.advisor_id = advisors.id \nWHERE advisors.first_name = 'Bill' AND advisors.last_name = 'Stropp';\n```"

In [37]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

system = """Double check the user's {dialect} query for common mistakes, including:
- Using NOT IN with NULL values
- Using UNION when UNION ALL should have been used
- Using BETWEEN for exclusive ranges
- Data type mismatch in predicates
- Properly quoting identifiers
- Using the correct number of arguments for functions
- Casting to the correct data type
- Using the proper columns for joins

If there are any of the above mistakes, rewrite the query.
If there are no mistakes, just reproduce the original query with no further commentary.

Output the final SQL query only."""
prompt = ChatPromptTemplate.from_messages(
    [("system", system), ("human", "{query}")]
).partial(dialect=db.dialect)
validation_chain = prompt | llm | StrOutputParser()

full_chain = {"query": chain} | validation_chain

In [61]:
system = """You are a {dialect} expert. Given an input question, create a syntactically correct {dialect} query to run.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per {dialect}. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Only use the following tables:
{table_info}

Write an initial draft of the query. Then double check the {dialect} query for common mistakes, including:
- Using NOT IN with NULL values
- Using UNION when UNION ALL should have been used
- Using BETWEEN for exclusive ranges
- Data type mismatch in predicates
- Properly quoting identifiers
- Using the correct number of arguments for functions
- Casting to the correct data type
- Using the proper columns for joins

Use format:

First draft: <<FIRST_DRAFT_QUERY>>
Final answer: <<FINAL_ANSWER_QUERY>>
"""
prompt = ChatPromptTemplate.from_messages(
    [("system", system), ("human", "{input}")]
).partial(dialect=db.dialect)


def parse_final_answer(output: str) -> str:
    return output.split("Final answer: ")[1]


chain = create_sql_query_chain(llm, db, prompt=prompt) | parse_final_answer
prompt.pretty_print()

================================ System Message ================================

You are a {dialect} expert. Given an input question, create a syntactically correct {dialect} query to run.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per {dialect}. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Only use the following tables:
{table_info}

Write an initial draft of the query. Then 

In [ ]:
query = chain.invoke(
    {
        "question": "Who are the top 5 highest rated advisors?"
    }
)
print(query)

In [122]:
def _modfify_message(query: str):

    SQL_PREFIX = """You are an agent designed to interact with a SQL database.
    Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
    Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
    You can order the results by a relevant column to return the most interesting examples in the database.
    Never query for all columns from a table. You must query only the columns that are needed to answer the question.
    Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
    Pay attention to use date('now') function to get the current date, if the question involves "today".

    Only use the following tables:
    {table_names}

    {table_info}
    
    Double check the {dialect} query for common mistakes, including:
    - Using NOT IN with NULL values
    - Using UNION when UNION ALL should have been used
    - Using BETWEEN for exclusive ranges
    - Data type mismatch in predicates
    - Properly quoting identifiers
    - Using the correct number of arguments for functions
    - Casting to the correct data type
    - Using the proper columns for joins

    You have access to tools for interacting with the database.

    Only use the below tools. Only use the information returned by the below tools to construct your final answer.
    You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

    DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

    To start you should ALWAYS look at the tables in the database to see what you can query.
    Do NOT skip this step.
    Then you should query the schema of the most relevant tables."""

    examples = example_selector.select_examples({"input": query})
    x = [('human', f"Use the following examples to guide yourself: \n {examples}")]
    prompt = ChatPromptTemplate.from_messages(
        [("system", SQL_PREFIX), ("placeholder", "{examples}")]
    )

    prompt = prompt.invoke(
        {
            "examples": x,
            "table_info": context["table_info"],
            "table_names": context["table_names"],
            "dialect": db.dialect,
        }
    ).to_messages()

    return prompt

In [121]:
_modfify_message("how many advisors")

[SystemMessage(content='You are an agent designed to interact with a SQL database.\n    Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.\n    Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.\n    You can order the results by a relevant column to return the most interesting examples in the database.\n    Never query for all columns from a table. You must query only the columns that are needed to answer the question.\n    Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.\n    Pay attention to use date(\'now\') function to get the current date, if the question involves "today".\n\n    Only use the following tables:\n    advisors, brokercheck_data, linkedin_profiles\n\n    \nCREATE TABLE advisors

In [49]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

llm = ChatOpenAI(model="gpt-4o")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
tools = toolkit.get_tools()

In [123]:
memory = SqliteSaver.from_conn_string(":memory:")
agent_executor = create_react_agent(llm, tools, messages_modifier=_modfify_message, checkpointer=memory)

In [126]:
def ask(query, config):
    return agent_executor.invoke( {"messages": query}, config)

In [127]:
question = "Tell me about billstroop?"
config = {
    "configurable": {
        "thread_id": 1434,
    }
}
res = ask(question, config)
print(res["messages"][-1])

TypeError: sequence item 0: expected str instance, list found

In [36]:
questions = ["Tell me more about Stephen Bryars", "How many advisors are there", "What palces did he work at", "Do they have any disclosure? If so how recent and what was the dollar amount?", "How do they work for?"]
responses = []
config = {
    "configurable": {
        "thread_id": 1234,
    }
}
for question in questions:
    responses.append(ask(question, config))